In [5]:
pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 122.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 129.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
#use RobustScaler for normalization
from sklearn.preprocessing import RobustScaler
# use for learning rate schedular
from tensorflow.keras.callbacks import ReduceLROnPlateau
#for early stopping
from tensorflow.keras.callbacks import EarlyStopping
#for l2 regularization
from tensorflow.keras.regularizers import l2
from collections import Counter
from sklearn.preprocessing import LabelEncoder

import time
import os

In [7]:

from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/diabetes_cleaned_dataset.csv')

Mounted at /content/drive


In [8]:
x = df.drop("Outcome", axis=1).values
y = df["Outcome"].values


In [9]:
scaler = RobustScaler()
x_scaled = scaler.fit_transform(x)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42, stratify=y)

In [11]:
scaler = RobustScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [12]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,125,33,0.627,50,1
1,1,85,66,29,125,26,0.351,31,0
2,8,183,64,29,125,23,0.672,32,1
3,1,89,66,23,94,28,0.167,21,0
4,0,137,40,35,168,43,2.288,33,1


In [13]:
model = Sequential([
    Dense(256, kernel_regularizer=l2(0.001), input_shape=(x_train.shape[1],)),  # Increased neurons
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.2),

    Dense(128, kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.2),

    Dense(64, kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.2),


    Dense(32, kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.2),

    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [14]:

optimizer = Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [26]:

history = model.fit(x_train, y_train, epochs=75, batch_size=50, validation_data=(x_test, y_test),callbacks=[early_stopping], verbose=1)


Epoch 1/75
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7560 - loss: 0.7502 - val_accuracy: 0.7208 - val_loss: 0.7887
Epoch 2/75
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7776 - loss: 0.7683 - val_accuracy: 0.7208 - val_loss: 0.7872
Epoch 3/75
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7793 - loss: 0.7439 - val_accuracy: 0.7208 - val_loss: 0.7870
Epoch 4/75
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7892 - loss: 0.7277 - val_accuracy: 0.7208 - val_loss: 0.7871
Epoch 5/75
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7923 - loss: 0.7464 - val_accuracy: 0.7273 - val_loss: 0.7875
Epoch 6/75
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7725 - loss: 0.7536 - val_accuracy: 0.7208 - val_loss: 0.7879
Epoch 7/75
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7443 - loss: 0.7670 - val_accuracy: 0.7273 - val_loss: 0.7889
Epoch 8/75
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7631 - loss: 0.7483 - val_accuracy: 0.7273 - v

In [27]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7090 - loss: 0.8055 
Test Accuracy: 72.08%


In [28]:
x_train_pca = np.array(x_train)
y_train = np.array(y_train)
train_loss, train_accuracy = model.evaluate(x_train_pca, y_train)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7904 - loss: 0.7332
Train Accuracy: 80.29%


In [29]:
y_pred = model.predict(x_test)
y_pred_labels = (y_pred > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred_labels))
print(classification_report(y_test, y_pred_labels))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Accuracy: 0.7207792207792207
              precision    recall  f1-score   support

           0       0.78      0.80      0.79       100
           1       0.61      0.57      0.59        54

    accuracy                           0.72       154
   macro avg       0.69      0.69      0.69       154
weighted avg       0.72      0.72      0.72       154



In [30]:

# Extract values from history
loss = history.history['loss']
val_loss = history.history['val_loss']
accuracy = history.history.get('accuracy', history.history.get('acc'))  # Some versions use 'acc'
val_accuracy = history.history.get('val_accuracy', history.history.get('val_acc'))

epochs = range(1, len(loss) + 1)


In [20]:
# Plot Loss
#plt.figure(figsize=(12, 5))
#plt.subplot(1, 2, 1)
#plt.plot(epochs, loss, 'bo-', label='Training Loss')
#plt.plot(epochs, val_loss, 'r*-', label='Validation Loss')
#plt.xlabel('Epochs')
#plt.ylabel('Loss')
#plt.title('Model Loss Over Epochs')
#plt.legend()


In [21]:
#plt.figure(figsize=(12, 4))
#plt.subplot(1, 2, 2)
#plt.plot(epochs, accuracy, 'bo-', label='Training Accuracy')
#plt.plot(epochs, val_accuracy, 'r*-', label='Validation Accuracy')
#lt.xlabel('Epochs')
#plt.ylabel('Accuracy')
#plt.title('Model Accuracy Over Epochs')
#plt.legend()

#plt.tight_layout()
#plt.show()

In [31]:
import tensorflow as tf
import os
import time
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.python.profiler import model_analyzer
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

def evaluate_model_metrics(model, x_test, y_test, model_name="model_metrics"):
    results = {}

    # Total Parameters
    results['total_parameters'] = model.count_params()

    # FLOPs Calculation
    func = tf.function(lambda x: model(x))
    concrete_func = func.get_concrete_function(
        tf.TensorSpec([1] + list(model.input_shape[1:]), model.inputs[0].dtype)
    )
    frozen_func = convert_variables_to_constants_v2(concrete_func)
    flops = model_analyzer.profile(
        graph=frozen_func.graph,
        options=ProfileOptionBuilder.float_operation()
    ).total_float_ops
    results['FLOPs'] = flops
    results['GFLOPs'] = flops / 1e9

    # Model Size
    model_path = f"{model_name}.h5"
    model.save(model_path)
    size_bytes = os.path.getsize(model_path)
    results['model_size_bytes'] = size_bytes
    results['model_size_MB'] = size_bytes / (1024 * 1024)

    # Inference Time
    start = time.time()
    model.predict(x_test)
    end = time.time()
    results['inference_time_seconds'] = end - start

    # Accuracy
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    results['accuracy'] = acc

    return results
evaluate_model_metrics(model, x_test, y_test, model_name="model_metrics")



5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


{'total_parameters': 47489,
 'FLOPs': 94497,
 'GFLOPs': 9.4497e-05,
 'model_size_bytes': 648936,
 'model_size_MB': 0.6188735961914062,
 'inference_time_seconds': 0.09905171394348145,
 'accuracy': 0.7207792401313782}